In [1]:
import pandas as pd
import tiktoken
import os
import numpy as np
from pathlib import Path
from ast import literal_eval

In [2]:
from openai import OpenAI
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "sk-proj-HSrHGyOHtEDtr9loLzRjT3BlbkFJTUFaqCLYDEs3B7qpBz7z"))
client = OpenAI()

In [5]:
transcript_and_translation_json= Path().cwd().parent.joinpath('data/output_json/transcript_and_translation_1.json')

In [6]:
transcript_and_translation_df = pd.read_json(transcript_and_translation_json)

In [58]:
false_text_df = pd.read_csv('embedded_false_statements.csv')

In [8]:
false_text_df

NameError: name 'false_text_df' is not defined

In [60]:
false_text_dict = false_text_df['statement_embedding'].to_dict()

In [61]:
false_text_df['statement_embedding'] = false_text_df.statement_embedding.apply(eval).apply(np.array)

In [62]:
false_text_df['statement_embedding'][0]

array([ 0.01533014, -0.00409743, -0.02695852, ...,  0.01104494,
        0.01542403, -0.01582639])

In [63]:
def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [64]:
# Semantic text search using embeddings

def cosine_similarity(vec1, vec2):
    # Normalize each vector to unit length
    vec1_norm = vec1 / np.linalg.norm(vec1)
    vec2_norm = vec2 / np.linalg.norm(vec2)
  
    # Calculate dot product between normalized vectors
    similarity = np.dot(vec1_norm, vec2_norm)
    return similarity

def search_false_statements(false_text_df, search_terms, n=3, pprint=True):
   embedding = get_embedding(search_terms, model='text-embedding-3-small')
   false_text_df['similarities'] = false_text_df.statement_embedding.apply(lambda x: cosine_similarity(x, embedding))
   results = (
      false_text_df.sort_values("similarities", ascending=False)
      .head(n).statement
   )
   if pprint:
      for r in results:
         print(r[:200])
         print()
   return results

search_terms = "Joe Biden took bribes"
results = search_false_statements(false_text_df, search_terms, n=3)

Says Burisma admitted in a Ukrainian court to paying Joe Biden $900,000.

James Biden "built the cages" and "(the Bidens) made millions from it."

"Vice President Biden threatened a foreign country and forced them to fire a prosecutor who was investigating a company," that paid his son Hunter.



In [65]:
false_text_df.sort_values('similarities',ascending=False).head(3).statement

1727     Says Burisma admitted in a Ukrainian court to ...
10431    James Biden "built the cages" and "(the Bidens...
8055     "Vice President Biden threatened a foreign cou...
Name: statement, dtype: object

In [66]:
false_text_df.sort_values('similarities',ascending=False)

,statement,statement_embedding,similarities
1727,Says Burisma admitted in a Ukrainian court to ...,"[-0.028112269937992096, -0.0002793215098790824...",0.568662
10431,"James Biden ""built the cages"" and ""(the Bidens...","[0.022143803536891937, 0.026470474898815155, 0...",0.538659
8055,"""Vice President Biden threatened a foreign cou...","[-0.0046212030574679375, 0.00427797669544816, ...",0.524526
10727,"Says Joe Biden is on video ""admitting to"" elec...","[0.012293009087443352, 0.011222226545214653, -...",0.522622
10808,"Says Joe Biden ""blackmailed Ukraine with a bil...","[0.0015891429502516985, -0.016753120347857475,...",0.518484
...,...,...,...
4738,"""Pathogens evolve to become less, not more, vi...","[0.014994660392403603, 0.039969805628061295, 0...",-0.010904
6545,It’s a myth that autism is genetic.,"[-0.0005327078979462385, 0.025457194074988365,...",-0.013286
11184,"Live Nation scheduled the 50,000-person Astrow...","[-0.03187582641839981, -0.017262330278754234, ...",-0.015338
9993,"""This unit is the only one in the state that s...","[-0.004030685871839523, 0.032830316573381424, ...",-0.021204
